In [2]:
# 必要なライブラリを読み込み
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [4]:
#前処理したデータを読み込み
#train = pd.read_csv('edited_train.csv').drop(["SOR"],axis = 1)
train = pd.read_csv('edited_train.csv').drop(["y", 'month', "year", 'hokkaido','kansai', 'disconfort', 'kantou', 'temperature', 'humidity',  'kyusyu','shikoku', 'tohoku', 'tyubu', 'tyugoku','stadium',"home area","away area"],axis = 1)
#test = pd.read_csv('edited_test.csv')
#data1.head(10)

In [5]:
drop_col = ["y","SOR","home_area","away_area","temperature","humidity","year"]
#drop_col = ["y","SOR"]
area_col = ["hokkaido","tohoku","kantou","tyubu","kansai","shikoku","tyugoku","kyusyu"]
#drop_col = ["y","SOR","home area","away area","temperature","humidity","year"]+area_col
drop_col = ["y","SOR","home area","away area","temperature","humidity","year","rerative distance"]

In [6]:
top6_index=train.corr()['logy'].abs().nlargest(15).index 
top6_index[1:]

Index(['stage', 'total_p_score', 'capa', 'distance(min)', 'home rank',
       'area rank', 'rerative distance', 'away rank', 'holiday',
       'weather rank', 'match day'],
      dtype='object')

In [17]:
# データの標準化
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
X = train.loc[:,top6_index[1:7]]
#X = train.drop(["y","SOR","home area","away area"],axis = 1)
y = train.loc[:,["logy"]].values.reshape(-1)

scaler = StandardScaler()
X_std = scaler.fit_transform(X) # Xを標準化した行列を生成
X_train, X_test, y_train, y_test = train_test_split(X_std, y, test_size = 0.25, random_state = 4649)

/Users/itarou/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/itarou/anaconda3/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [7]:
X.columns

Index(['stage', 'total_p_score', 'capa', 'distance(min)', 'home rank',
       'area rank', 'rerative distance', 'away rank', 'holiday'],
      dtype='object')

In [8]:
from sklearn.model_selection import StratifiedKFold
kf_5 = StratifiedKFold(n_splits=2, shuffle=True, random_state=0)

In [9]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
params = {"alpha":np.logspace(-2, 4, 24)}
# (モデルのインスタンス, 試したいパラメータの値, 分割方法)
gs_rd = GridSearchCV(SVR(), params,cv = 5)

In [10]:
np.logspace(-2, 4, 10)

array([1.00000000e-02, 4.64158883e-02, 2.15443469e-01, 1.00000000e+00,
       4.64158883e+00, 2.15443469e+01, 1.00000000e+02, 4.64158883e+02,
       2.15443469e+03, 1.00000000e+04])

In [ ]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
# ハイパーパラメータのチューニング
params_cnt = 5
cstart = 5
cstop = 10
gstart = -7
gstop =-3
#params = {"C":np.logspace(10,100,params_cnt), "epsilon":np.logspace(-1,1,params_cnt)}
#params={'C': np.logspace(cstart, cstop, params_cnt),'gamma': np.logspace(gstart, gstop, params_cnt)}
params={'C':np.logspace(1, 5, 4),'gamma': np.logspace(1, 5, 2)}
gridsearch = GridSearchCV(SVR(kernel="linear"), params, cv=2, scoring="r2", return_train_score=True)
gridsearch.fit(X_train, y_train)
print("C, εのチューニング")
print("最適なパラメーター =", gridsearch.best_params_)
print("精度 =", gridsearch.best_score_)
print()

In [21]:
paramd = {'C': 4.6415888336127775, 'gamma': 0.0001}


In [22]:
from sklearn.svm import SVR
#regr = SVR(kernel="linear", C=gridsearch.best_params_["C"], epsilon=gridsearch.best_params_["gamma"])
regr = SVR(kernel="linear", C=paramd["C"], epsilon=paramd["gamma"])
regr.fit(X_train, y_train)
print("切片と係数")
print(regr.intercept_)
print(regr.coef_)
print()
# テストデータの精度を計算
print("テストデータにフィット")
print("テストデータの精度 =", regr.score(X_test, y_test))
print()
print("※参考")
print("訓練データの精度 =", regr.score(X_train, y_train))
#print("交差検証データの精度 =", regr.score(X_norm[valid_indices, :], y[valid_indices]))

切片と係数
[9.03801087]
[[ 0.30209176  0.07631145  0.20229433 -0.06693325 -0.1781669  -0.09771627]]

テストデータにフィット
テストデータの精度 = 0.6937377288979751

※参考
訓練データの精度 = 0.7015256966688195


In [23]:
from sklearn.metrics import mean_squared_error as mse
print('train RMSE:%.3f'%(mse(np.exp(y_train), np.exp(regr.predict(X_train))) ** (1/2)))

train RMSE:4758.466


In [24]:
from sklearn.metrics import mean_squared_error as mse

print('train RMSE:%.3f'%(mse(np.exp(y_test), np.exp(regr.predict(X_test))) ** (1/2)))

train RMSE:4389.611
